# Cross-Match KOA and Megafile

This code cross-matches the file that contains all the information for the HGCA stars, or the 'megafile', and the file which contains all the observations that have been reduced in KOA.

Because the megafile stars are by definition accelerating, it is important that the RA and Dec of these files is converted to be correct for the date of the Keck observations. Because it would take to long to convert every RA to every line in the megafile we only convert by each date in the megafile and then check the stars en masse. 

For the observation time in the megafile we use the average epoch of the Gaia RA and Dec. 

In [22]:
# Necessary Modules 
import pandas as pd
from astropy.coordinates import SkyCoord, Distance
from astropy import units as u
from astropy.time import Time
import numpy as np
import time
import glob

In [23]:
# Reading in the HGCA megafile
megafile = pd.read_csv('/Users/Jess/HGCA_survey_paper/megafile.csv')
print('There are ' + str(len(megafile)) + ' HCGA stars.')

There are 115663 HCGA stars.


In [24]:
# Reading in KOA file
koafile = pd.read_csv('/Users/Jess/HGCA_survey_paper/new_keck_stars_edited.csv')
print('There are ' + str(len(koafile)) + ' files in the KOA database.')

There are 335428 files in the KOA database.


In [25]:
# Create a list of unique observation dates
myset = set(koafile['date'])
myset = list(myset)
print('The KOA list of files has ' + str(len(koafile['date'])) + ' files, which have ' + str(len(myset)) + ' unique observation dates.')

The KOA list of files has 335428 files, which have 1782 unique observation dates.


In [26]:
# Determine the average epoch time
ave_time = (megafile['epoch_ra_gaia'] + megafile['epoch_dec_gaia']) / 2

In [27]:
# Set up catalog of all the stars in the megafile
c = SkyCoord(ra = list(megafile['ra'])*u.deg, dec = list(megafile['dec'])*u.deg, 
distance = Distance(parallax = list(megafile['gaia_parallax']) * u.mas), 
pm_ra_cosdec = list(megafile['pmra_gaia']) * u.mas/u.yr, 
pm_dec = list(megafile['pmdec_gaia']) * u.mas/u.yr, 
obstime = Time(ave_time, format='decimalyear'))
print(len(c))

115663


In [28]:
# Breaking down the KOA file into groups by date
a = 0
for i in myset:
    df = koafile[koafile["date"] == i]
    df.to_csv('/Users/Jess/HGCA_survey_paper/koa_date/koa_date' + str(a) + '.csv')
    a = a + 1

In [36]:
# Setting timer for segment
t0 = time.time()

# list to save final indexes
total_indices = []
filename_indices = []
file_names = []

# Opening each of the files - there is one for each date, each containing all observations at that date. 
koa_dates = glob.glob('/Users/Jess/HGCA_survey_paper/koa_date/koa_date*.csv')

# Loop through each date 
for i in koa_dates:
    koad = pd.read_csv(i) # Date file generally 
    print('There are ' + str(len(koad)) + ' KOA observations with this date.')
    
    koadd = pd.read_csv(i)['date'][0] # The specific date in this file
    print('Shifting HGCA stars to date No#: ' + str(koadd)) 
    
    # Set up catalog of all the stars in the megafile
    c = SkyCoord(ra = list(megafile['ra'])*u.deg, dec = list(megafile['dec'])*u.deg, 
    distance = Distance(parallax = list(megafile['gaia_parallax']) * u.mas), 
    pm_ra_cosdec = list(megafile['pmra_gaia']) * u.mas/u.yr, 
    pm_dec = list(megafile['pmdec_gaia']) * u.mas/u.yr, 
    obstime = Time(ave_time, format='decimalyear'))
   
    # Shift the megafile to the new observation date
    sc = c.apply_space_motion(new_obstime=Time(koadd))
        
    # Create a catalog for the KOA observations at this date, removing the 'NaN' values
    catalog = SkyCoord(ra=koad['ra']*u.degree, dec=koad['dec']*u.degree)
    clean = (~np.isnan(catalog.ra) & ~np.isnan(catalog.dec))
    catalog_clean = catalog[clean]
        
    # Search around the sky 
    idxcatalog, idxsc, d2d, d3d = sc.search_around_sky(catalog_clean, 20*u.arcsecond)
    print('There are ' + str(len(set(idxsc))) + ' stars in HGCA that are also present in KOA observations. \n')
    
    # Save indices for matches found
    for x1 in idxsc:
        total_indices.append(x1)

# Complete timer
t1 = time.time()
total = t1-t0
print('The time to run this code segment is ' + str(total/60) + ' minutes.')

There are 18 KOA observations with this date.
Shifting HGCA stars to date No#: 2012-10-24
There are 2 stars in HGCA that are also present in KOA observations. 

There are 316 KOA observations with this date.
Shifting HGCA stars to date No#: 2014-05-09
There are 2 stars in HGCA that are also present in KOA observations. 

There are 161 KOA observations with this date.
Shifting HGCA stars to date No#: 2009-09-10
There are 1 stars in HGCA that are also present in KOA observations. 

There are 745 KOA observations with this date.
Shifting HGCA stars to date No#: 2012-07-27
There are 3 stars in HGCA that are also present in KOA observations. 

There are 554 KOA observations with this date.
Shifting HGCA stars to date No#: 2013-11-15
There are 2 stars in HGCA that are also present in KOA observations. 

There are 45 KOA observations with this date.
Shifting HGCA stars to date No#: 2005-07-01
There are 0 stars in HGCA that are also present in KOA observations. 

There are 1 KOA observations w

There are 0 stars in HGCA that are also present in KOA observations. 

There are 13 KOA observations with this date.
Shifting HGCA stars to date No#: 2018-02-03
There are 0 stars in HGCA that are also present in KOA observations. 

There are 373 KOA observations with this date.
Shifting HGCA stars to date No#: 2012-07-22
There are 6 stars in HGCA that are also present in KOA observations. 

There are 193 KOA observations with this date.
Shifting HGCA stars to date No#: 2012-03-28
There are 0 stars in HGCA that are also present in KOA observations. 

There are 99 KOA observations with this date.
Shifting HGCA stars to date No#: 2013-03-24
There are 2 stars in HGCA that are also present in KOA observations. 

There are 408 KOA observations with this date.
Shifting HGCA stars to date No#: 2009-09-22
There are 0 stars in HGCA that are also present in KOA observations. 

There are 50 KOA observations with this date.
Shifting HGCA stars to date No#: 2017-07-24
There are 1 stars in HGCA that 

There are 1 stars in HGCA that are also present in KOA observations. 

There are 105 KOA observations with this date.
Shifting HGCA stars to date No#: 2013-09-19
There are 0 stars in HGCA that are also present in KOA observations. 

There are 423 KOA observations with this date.
Shifting HGCA stars to date No#: 2010-04-03
There are 3 stars in HGCA that are also present in KOA observations. 

There are 1 KOA observations with this date.
Shifting HGCA stars to date No#: 2017-06-03
There are 0 stars in HGCA that are also present in KOA observations. 

There are 646 KOA observations with this date.
Shifting HGCA stars to date No#: 2015-12-05
There are 2 stars in HGCA that are also present in KOA observations. 

There are 96 KOA observations with this date.
Shifting HGCA stars to date No#: 2005-10-29
There are 0 stars in HGCA that are also present in KOA observations. 

There are 99 KOA observations with this date.
Shifting HGCA stars to date No#: 2004-02-05
There are 0 stars in HGCA that a

There are 0 stars in HGCA that are also present in KOA observations. 

There are 142 KOA observations with this date.
Shifting HGCA stars to date No#: 2010-07-14
There are 1 stars in HGCA that are also present in KOA observations. 

There are 143 KOA observations with this date.
Shifting HGCA stars to date No#: 2011-08-27
There are 0 stars in HGCA that are also present in KOA observations. 

There are 297 KOA observations with this date.
Shifting HGCA stars to date No#: 2019-06-28
There are 0 stars in HGCA that are also present in KOA observations. 

There are 348 KOA observations with this date.
Shifting HGCA stars to date No#: 2017-06-28
There are 1 stars in HGCA that are also present in KOA observations. 

There are 23 KOA observations with this date.
Shifting HGCA stars to date No#: 2007-09-21
There are 0 stars in HGCA that are also present in KOA observations. 

There are 647 KOA observations with this date.
Shifting HGCA stars to date No#: 2012-01-04
There are 2 stars in HGCA tha

There are 0 stars in HGCA that are also present in KOA observations. 

There are 496 KOA observations with this date.
Shifting HGCA stars to date No#: 2014-10-12
There are 5 stars in HGCA that are also present in KOA observations. 

There are 109 KOA observations with this date.
Shifting HGCA stars to date No#: 2013-02-25
There are 0 stars in HGCA that are also present in KOA observations. 

There are 125 KOA observations with this date.
Shifting HGCA stars to date No#: 2006-05-22
There are 0 stars in HGCA that are also present in KOA observations. 

There are 103 KOA observations with this date.
Shifting HGCA stars to date No#: 2003-12-25
There are 1 stars in HGCA that are also present in KOA observations. 

There are 407 KOA observations with this date.
Shifting HGCA stars to date No#: 2009-11-01
There are 14 stars in HGCA that are also present in KOA observations. 

There are 14 KOA observations with this date.
Shifting HGCA stars to date No#: 2009-06-08
There are 0 stars in HGCA th

...

There are 1 stars in HGCA that are also present in KOA observations. 

There are 70 KOA observations with this date.
Shifting HGCA stars to date No#: 2013-08-01
There are 0 stars in HGCA that are also present in KOA observations. 

There are 3 KOA observations with this date.
Shifting HGCA stars to date No#: 2016-08-06
There are 0 stars in HGCA that are also present in KOA observations. 

There are 91 KOA observations with this date.
Shifting HGCA stars to date No#: 2010-07-13
There are 1 stars in HGCA that are also present in KOA observations. 

There are 32 KOA observations with this date.
Shifting HGCA stars to date No#: 2006-06-29
There are 0 stars in HGCA that are also present in KOA observations. 

There are 5 KOA observations with this date.
Shifting HGCA stars to date No#: 2017-07-04
There are 0 stars in HGCA that are also present in KOA observations. 

There are 4 KOA observations with this date.
Shifting HGCA stars to date No#: 2009-04-14
There are 2 stars in HGCA that are al

There are 14 stars in HGCA that are also present in KOA observations. 

There are 156 KOA observations with this date.
Shifting HGCA stars to date No#: 2002-08-21
There are 5 stars in HGCA that are also present in KOA observations. 

There are 652 KOA observations with this date.
Shifting HGCA stars to date No#: 2014-09-03
There are 10 stars in HGCA that are also present in KOA observations. 

There are 174 KOA observations with this date.
Shifting HGCA stars to date No#: 2010-04-05
There are 2 stars in HGCA that are also present in KOA observations. 

There are 100 KOA observations with this date.
Shifting HGCA stars to date No#: 2003-05-23
There are 3 stars in HGCA that are also present in KOA observations. 

There are 215 KOA observations with this date.
Shifting HGCA stars to date No#: 2011-07-20
There are 0 stars in HGCA that are also present in KOA observations. 

There are 9 KOA observations with this date.
Shifting HGCA stars to date No#: 2007-12-11
There are 1 stars in HGCA th

There are 0 stars in HGCA that are also present in KOA observations. 

There are 161 KOA observations with this date.
Shifting HGCA stars to date No#: 2015-08-12
There are 0 stars in HGCA that are also present in KOA observations. 

There are 261 KOA observations with this date.
Shifting HGCA stars to date No#: 2005-10-11
There are 1 stars in HGCA that are also present in KOA observations. 

There are 2 KOA observations with this date.
Shifting HGCA stars to date No#: 2018-03-24
There are 0 stars in HGCA that are also present in KOA observations. 

There are 475 KOA observations with this date.
Shifting HGCA stars to date No#: 2007-10-25
There are 13 stars in HGCA that are also present in KOA observations. 

There are 12 KOA observations with this date.
Shifting HGCA stars to date No#: 2009-10-01
There are 0 stars in HGCA that are also present in KOA observations. 

There are 149 KOA observations with this date.
Shifting HGCA stars to date No#: 2016-07-27
There are 1 stars in HGCA that

There are 3 stars in HGCA that are also present in KOA observations. 

There are 9 KOA observations with this date.
Shifting HGCA stars to date No#: 2011-12-28
There are 0 stars in HGCA that are also present in KOA observations. 

There are 74 KOA observations with this date.
Shifting HGCA stars to date No#: 2012-09-06
There are 0 stars in HGCA that are also present in KOA observations. 

There are 602 KOA observations with this date.
Shifting HGCA stars to date No#: 2017-11-05
There are 9 stars in HGCA that are also present in KOA observations. 

There are 327 KOA observations with this date.
Shifting HGCA stars to date No#: 2019-07-01
There are 0 stars in HGCA that are also present in KOA observations. 

There are 157 KOA observations with this date.
Shifting HGCA stars to date No#: 2017-04-06
There are 0 stars in HGCA that are also present in KOA observations. 

There are 1 KOA observations with this date.
Shifting HGCA stars to date No#: 2017-05-19
There are 0 stars in HGCA that ar

There are 0 stars in HGCA that are also present in KOA observations. 

There are 161 KOA observations with this date.
Shifting HGCA stars to date No#: 2007-08-02
There are 2 stars in HGCA that are also present in KOA observations. 

There are 432 KOA observations with this date.
Shifting HGCA stars to date No#: 2018-08-22
There are 0 stars in HGCA that are also present in KOA observations. 

There are 155 KOA observations with this date.
Shifting HGCA stars to date No#: 2012-05-18
There are 1 stars in HGCA that are also present in KOA observations. 

There are 1 KOA observations with this date.
Shifting HGCA stars to date No#: 2017-11-03
There are 0 stars in HGCA that are also present in KOA observations. 

There are 343 KOA observations with this date.
Shifting HGCA stars to date No#: 2009-12-05
There are 2 stars in HGCA that are also present in KOA observations. 

There are 293 KOA observations with this date.
Shifting HGCA stars to date No#: 2011-08-23
There are 0 stars in HGCA that

In [33]:
# Matching the indices to rows in the megafile and saving as a new csv.
print('There are ' + str(len(set(total_indices))) + ' unique stars in this list.')
idx_set = list(set(total_indices))
cross_matched_stars = megafile.iloc[idx_set]
cross_matched_stars.to_csv('/Users/Jess/HGCA_survey_paper/completed_cross_match.csv')

[array([111769, 111769, 111769, 111769, 111769, 111769, 111769, 111769,
       111769, 111769, 111769, 111769, 111769, 111769, 111769, 111775,
       111769, 111769, 111769]), array([76400, 76400, 76400, 76400, 76400, 76400, 76400, 76400, 76400,
       71275, 71275, 71275, 71275, 71275, 71275, 71275, 71275, 71275,
       71275, 71275, 71275, 71275, 71275, 71275, 71275, 71275, 71275,
       71275, 71275, 71275, 71275]), array([101474, 101474, 101474, 101474, 101474, 101474, 101474, 101474,
       101474, 101474, 101474, 101474, 101474, 101474, 101474, 101474,
       101474, 101474, 101474, 101474, 101474]), array([ 88103,  88103,  88103,  88103,  88103,  88103,  88103,  88103,
        88103,  88103,  88103,  88103,  88103,  88103,  88103,  88103,
        88103,  88103,  88103,  88103,  88103,  88103,  88103,  88103,
        88103,  88103,  88103,  88103,  88103,  95428,  95428,  95428,
        95428,  95428,  95428,  95428,  95428,  95428,  95428,  95428,
        95428,  95428,  95428, 

TypeError: unhashable type: 'numpy.ndarray'

In [10]:
print(len(filename_indices))

114271


In [20]:
# Determine the cut down file names for each of the 'successful' KOA observations and save into a new file.
idx_set_koa = list(set(filename_indices))
print(len(idx_set_koa))
koa_stars = koafile.iloc[idx_set_koa]
print(len(koa_stars))
filenames = list(koa_stars['File'])

filenames_cut = []
for i in filenames:
    new_filename = i[51:-10]
    filenames_cut.append(new_filename)

    
col = zip(filenames_cut)
header = ['new_filenames']
Fnames = pd.DataFrame(col, columns=header)
Fnames.to_csv('/Users/Jess/HGCA_survey_paper/koa_shortened_names.csv', index=False)

1658
1658
20010730.48358 20011219.51483


In [12]:
fnames = pd.read_csv('/Users/Jess/HGCA_survey_paper/koa_shortened_names.csv')

In [13]:
print(len(fnames))

1658
